In [ ]:
!pip install tonic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for importRosbag: filename=importRosbag-1.0.3-py3-none-any.whl size=25466 sha256=ea11726e8e1150dedfffdddc41be65155b8c1514bff67457e046ec53bc5d875b
  Stored in directory: /root/.cache/pip/wheels/7f/a3/dd/959c5e661d227bac3fe3988df383132b18506f39831ffb2209
Successfully built importRosbag


In [ ]:
pip install tonic --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import urllib.request
urllib.request.urlretrieve('https://raw.githubusercontent.com/jeshraghian/QSNNs/main/requirements.txt', 'requirements.txt')
!pip install -r requirements.txt --quiet
import torch, torch.nn as nn
import snntorch as snn
import brevitas.nn as qnn
from snntorch.spikevision import spikedata


def load_data(config):
    data_dir = config["data_dir"]
    # Note: the train set / test set are of different durations, we used num_steps=100 due to memory limits.
    # You will likely to improve our reported results by increasing num_steps=100 to 150.
    trainset = spikedata.DVSGesture(data_dir, train=True, num_steps=100, dt=3000, ds=4)
    testset = spikedata.DVSGesture(data_dir, train=False, num_steps=600, dt=3000, ds=4)
    return trainset, testset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.4/411.4 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.8 MB/s eta 0:00:00


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
<ipython-input-3-94648b2f701f>:7: DeprecationWarning: The module snntorch.spikevision is deprecated. For loading neuromorphic datasets, we recommend using the Tonic project: https://github.com/neuromorphs/tonic
  from snntorch.spikevision import spikedata


In [ ]:
def train(config, net, trainloader, criterion, optimizer, device="cpu", scheduler=None):
    net.train()
    loss_accum = []
    lr_accum = []
    i = 0
    for data, labels in trainloader:
        data, labels = data.to(device), labels.to(device)
        spk_rec, _ = net(data.permute(1, 0, 2, 3, 4))
        loss = criterion(spk_rec, labels.long())
        optimizer.zero_grad()
        loss.backward()
        if config["grad_clip"]:
            nn.utils.clip_grad_norm_(net.parameters(), 1.0)

        if config["weight_clip"]:
            with torch.no_grad():
                for param in net.parameters():
                    param.clamp_(-1, 1)

        optimizer.step()
        scheduler.step()
        loss_accum.append(loss.item() / config["num_steps"])
        lr_accum.append(optimizer.param_groups[0]["lr"])

    return loss_accum, lr_accum


In [ ]:
import torch
from snntorch import functional as SF


def test(config, net, testloader, device="cpu"):
    correct = 0
    total = 0
    with torch.no_grad():
        net.eval()
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs, _ = net(images.permute(1, 0, 2, 3, 4))
            accuracy = SF.accuracy_rate(outputs, labels.long())
            total += labels.size(0)
            correct += accuracy * labels.size(0)

    return 100 * correct / total


In [ ]:
import torch


class EarlyStopping_acc:
    """Early stops the training if test acc doesn't improve after a given patience."""

    def __init__(
        self, patience=7, verbose=False, delta=0, path="checkpoint.pt", trace_func=print
    ):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.test_loss_min = 0
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, test_loss, model):
        score = test_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(test_loss, model)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            self.trace_func(
                f"EarlyStopping counter: {self.counter} out of {self.patience}"
            )
            if self.counter >= self.patience:
                self.early_stop = True
                self.counter = 0
        else:
            self.best_score = score
            self.save_checkpoint(test_loss, model)
            self.counter = 0

    def save_checkpoint(self, test_loss, model):
        """Saves model when test acc increases."""
        if self.verbose:
            self.trace_func(
                f"Test acc increased ({self.test_loss_min:.6f} --> {test_loss:.6f}).  Saving model ..."
            )

        torch.save(model.state_dict(), self.path)
        self.test_loss_min = test_loss


In [ ]:

def set_all_seeds(seed=0):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [ ]:
import snntorch as snn
from snntorch import functional as SF
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import time



def evaluate(Net, config, load_data, train, test, optim_func):
    file_name = config["exp_name"]
    for trial in range(config["num_trials_eval"]):
        csv_name = file_name + "_t" + str(trial) + ".csv"
        model_name = file_name + "_t" + str(trial) + ".pt"
        num_epochs = config["num_epochs_eval"]
        set_all_seeds(config["seed"] + trial)
        df_train_loss = pd.DataFrame()
        df_test_acc = pd.DataFrame(columns=["epoch", "test_acc", "train_time"])
        df_lr = pd.DataFrame()
        # Initialize the network
        net = Net(config)
        device = "cpu"
        if torch.cuda.is_available():
            device = "cuda"

        net.to(device)
        # Initialize the optimizer and scheduler
        criterion = SF.mse_count_loss(
            correct_rate=config["correct_rate"], incorrect_rate=config["incorrect_rate"]
        )
        optimizer, scheduler, loss_dependent = optim_func(net, config)
        # Early stopping condition
        if config["early_stopping"]:
            early_stopping = EarlyStopping_acc(
                patience=config["patience"], verbose=True, path=model_name
            )
            early_stopping.early_stop = False
            early_stopping.best_score = None

        # Load data
        trainset, testset = load_data(config)
        config["dataset_length"] = len(trainset)
        trainloader = DataLoader(
            trainset, batch_size=int(config["batch_size"]), shuffle=True
        )
        testloader = DataLoader(
            testset, batch_size=int(config["batch_size"]), shuffle=False
        )
        if loss_dependent:
            old_loss_hist = float("inf")

        print(
            f"=======Trial: {trial}, Batch: {config['batch_size']}, beta: {config['beta']:.3f}, threshold: {config['threshold']:.2f}, slope: {config['slope']}, lr: {config['lr']:.3e}======"
        )
        # Train
        for epoch in range(num_epochs):
            start_time = time.time()
            loss_list, lr_list = train(
                config, net, trainloader, criterion, optimizer, device, scheduler
            )
            epoch_time = time.time() - start_time
            if loss_dependent:
                avg_loss_hist = sum(loss_list) / len(loss_list)
                if avg_loss_hist > old_loss_hist:
                    for param_group in optimizer.param_groups:
                        param_group["lr"] = param_group["lr"] * 0.5
                else:
                    old_loss_hist = avg_loss_hist

            # Test
            test_accuracy = test(config, net, testloader, device)
            print(f"Epoch: {epoch} \tTest Accuracy: {test_accuracy}")
            df_lr = df_lr.append(lr_list, ignore_index=True)

            df_train_loss = df_train_loss.append(loss_list, ignore_index=True)
            df_test_acc = df_test_acc.append(
                {"epoch": epoch, "test_acc": test_accuracy, "train_time": epoch_time},
                ignore_index=True,
            )
            if config["save_csv"]:
                df_train_loss.to_csv("loss_" + csv_name, index=False)
                df_test_acc.to_csv("acc_" + csv_name, index=False)
                df_lr.to_csv("lr_" + csv_name, index=False)

            if config["early_stopping"]:
                early_stopping(test_accuracy, net)
                if early_stopping.early_stop:
                    print("Early stopping")
                    early_stopping.early_stop = False
                    early_stopping.best_score = None
                    break


In [ ]:
import snntorch as snn
from snntorch import surrogate
import torch
import torch.nn as nn
import torch.nn.functional as F
import brevitas.nn as qnn


class Net(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_bits = config["num_bits"]
        self.thr = config["threshold"]
        self.slope = config["slope"]
        self.beta = config["beta"]
        self.num_steps = config["num_steps"]
        self.batch_norm = config["batch_norm"]
        self.p1 = config["dropout"]
        self.spike_grad = surrogate.fast_sigmoid(self.slope)
        if self.num_bits is None:
            self.init_net()
        else:
            self.init_quantized_net()

    def init_net(self):
        self.conv1 = nn.Conv2d(2, 16, 5, bias=False)
        self.conv1_bn = nn.BatchNorm2d(16)
        self.lif1 = snn.Leaky(self.beta, threshold=self.thr, spike_grad=self.spike_grad)
        self.conv2 = nn.Conv2d(16, 32, 5, bias=False)
        self.conv2_bn = nn.BatchNorm2d(32)
        self.lif2 = snn.Leaky(self.beta, threshold=self.thr, spike_grad=self.spike_grad)
        self.fc1 = nn.Linear(32 * 5 * 5, 11)
        self.lif3 = snn.Leaky(self.beta, threshold=self.thr, spike_grad=self.spike_grad)
        self.dropout = nn.Dropout(self.p1)

    def init_quantized_net(self):
        self.conv1 = qnn.QuantConv2d(
            2, 16, 5, bias=False, weight_bit_width=self.num_bits
        )
        self.conv1_bn = nn.BatchNorm2d(16)
        self.lif1 = snn.Leaky(self.beta, threshold=self.thr, spike_grad=self.spike_grad)
        self.conv2 = qnn.QuantConv2d(
            16, 32, 5, bias=False, weight_bit_width=self.num_bits
        )
        self.conv2_bn = nn.BatchNorm2d(32)
        self.lif2 = snn.Leaky(self.beta, threshold=self.thr, spike_grad=self.spike_grad)
        self.fc1 = qnn.QuantLinear(
            32 * 5 * 5, 11, bias=False, weight_bit_width=self.num_bits
        )
        self.lif3 = snn.Leaky(self.beta, threshold=self.thr, spike_grad=self.spike_grad)
        self.dropout = nn.Dropout(self.p1)

    def forward(self, x):
        # Initialize hidden states and outputs at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        # Record the final layer
        spk3_rec = []
        mem3_rec = []
        for step in range(x.size(0)):
            cur1 = F.avg_pool2d(self.conv1(x[step]), 2)
            if self.batch_norm:
                cur1 = self.conv1_bn(cur1)

            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = F.avg_pool2d(self.conv2(spk1), 2)
            if self.batch_norm:
                cur2 = self.conv2_bn(cur2)

            spk2, mem2 = self.lif2(cur2, mem2)
            cur3 = self.dropout(self.fc1(spk2.flatten(1)))
            spk3, mem3 = self.lif3(cur3, mem3)
            spk3_rec.append(spk3)
            mem3_rec.append(mem3)

        return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)


In [ ]:
import torch
import sys
import os
import random
import numpy as np



config = {
    "exp_name": "DVS",  # Experiment name
    "num_trials_eval": 3,  # Number of trails to execute (separate training and evaluation instances)
    "num_epochs_eval": 500,  # Number of epochs to train for (per trial)
    "data_dir": "~/data/",  # Data directory to download and store data
    "batch_size": 16,  # Batch size
    "seed": 0,  # Random seed
    "num_workers": 0,  # Number of workers for the dataloader
    "num_bits": 4,  # Bit resolution. If None, floating point resolution is used
    "save_csv": True,  # Whether or not to save loss, lr, and accuracy dataframes
    "early_stopping": True,  # Whether or not to use early stopping
    "patience": 100,  # Number of epochs to wait for improvement before stopping
    # Network parameters
    "grad_clip": True,  # Whether or not to clip gradients
    "weight_clip": True,  # Whether or not to clip weights
    "batch_norm": False,  # Whether or not to use batch normalization
    "dropout": 0.203,  # Dropout rate
    "beta": 0.614,  # Decay rate parameter (beta)
    "threshold": 0.427,  # Threshold parameter (theta)
    "lr": 2.634e-3,  # Initial learning rate
    "slope": 4.413,  # Slope value (k)
    # Fixed params
    "num_steps": 1,  # Number of timesteps to encode input for 100 TODO
    "correct_rate": 0.8,  # Correct rate
    "incorrect_rate": 0.2,  # Incorrect rate
    "betas": (0.9, 0.999),  # Adam optimizer beta values
    "t_max": 735,  # Frequency of the cosine annealing scheduler (5 epochs)
    "t_0": 735,  # Initial frequency of the cosine annealing scheduler
    "t_mult": 2,  # The frequency of cosine is halved after every 4690 iters (10 epochs)
    "eta_min": 0,  # Minimum learning rate
}


def optim_func(net, config):
    optimizer = torch.optim.Adam(
        net.parameters(), lr=config["lr"], betas=config["betas"]
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=config["t_0"], eta_min=config["eta_min"], last_epoch=-1
    )
    loss_dependent = False
    return optimizer, scheduler, loss_dependent


if __name__ == "__main__":
    evaluate(Net, config, load_data, train, test, optim_func)


AttributeError: ignored